In [72]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()

In [73]:
import geemap
Map = geemap.Map()

In [110]:
terracl = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filterDate('2000-01-01','2020-12-31').select(['tmmn','tmmx'])

In [116]:
# create empty image
myimg = ee.Image().rename('b_01')
# create new image collection based on long-term averages
for i in range(1,12+1):
    # average for a given month across the whole period
    monthi = terracl.filter(ee.Filter.calendarRange(i,i,'month')).mean()
    # calculate mean, rescale and add month as new property
    avgtemp = monthi.expression('((tmmn + tmmx)/2)*0.1',{'tmmn':monthi.select('tmmn'),'tmmx':monthi.select('tmmx')}).rename(['b_'+str(i).zfill(2)]).set(ee.Dictionary({'Month': str(i).zfill(2)}))
    myimg = myimg.addBands(avgtemp,overwrite = True)

In [117]:
myimg1 = myimg.reproject(ee.Projection('EPSG:4326').scale(0.05,0.05))

In [118]:
myimg1.propertyNames().getInfo()

['system:bands', 'system:band_names']

In [109]:
task = ee.batch.Export.image.toAsset(image = myimg1,description = 'terraclimate_months',
                              assetId='users/marcogirardello/phenoutils/terraclimate_months',
                             crs = 'EPSG:4326',scale = 5565.974539663679,maxPixels=1e13)
task.start()